<a href="https://colab.research.google.com/github/CatalystM47/Deep_Learning/blob/main/sign_MNIST_Rotate180.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Augmentation

In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

In [ ]:
print("Number of training data : ", len(train_df))
print("Number of test data : ", len(valid_df))

Number of training data :  7255
Number of test data :  7172


In [ ]:
y_train = train_df["label"]
y_valid = valid_df["label"]

In [ ]:
print(y_train)
print(y_valid)

0        3
1        6
2        2
3        2
4       12
        ..
7250     9
7251    14
7252     1
7253    23
7254     9
Name: label, Length: 7255, dtype: int64
0        6
1        5
2        9
3        0
4        3
        ..
7167     1
7168    11
7169     2
7170     4
7171     2
Name: label, Length: 7172, dtype: int64


In [ ]:
del train_df["label"]
del valid_df["label"]

x_train = train_df.values
x_valid = valid_df.values

In [ ]:
print(x_train)
print(x_valid)

[[107. 118. 127. ... 204. 203. 202.]
 [155. 157. 156. ... 103. 135. 149.]
 [187. 188. 188. ... 195. 194. 195.]
 ...
 [ 59.  93. 111. ... 168. 176. 165.]
 [179. 182. 185. ... 176. 199. 218.]
 [ 66.  80.  87. ...  nan  nan  nan]]
[[149 149 150 ... 112 120 107]
 [126 128 131 ... 184 182 180]
 [ 85  88  92 ... 225 224 222]
 ...
 [190 191 190 ... 211 209 208]
 [201 205 208 ...  67  70  63]
 [173 174 173 ... 195 193 192]]


In [ ]:
x_train2 = np.flip(x_train)

In [ ]:
x_train2

array([[ nan,  nan,  nan, ...,  87.,  80.,  66.],
       [218., 199., 176., ..., 185., 182., 179.],
       [165., 176., 168., ..., 111.,  93.,  59.],
       ...,
       [195., 194., 195., ..., 188., 188., 187.],
       [149., 135., 103., ..., 156., 157., 155.],
       [202., 203., 204., ..., 127., 118., 107.]])

In [ ]:
apeend_train = np.vstack([x_train, x_train2])

In [ ]:
len(apeend_train)

14510

In [ ]:
apeend_train

array([[107., 118., 127., ..., 204., 203., 202.],
       [155., 157., 156., ..., 103., 135., 149.],
       [187., 188., 188., ..., 195., 194., 195.],
       ...,
       [195., 194., 195., ..., 188., 188., 187.],
       [149., 135., 103., ..., 156., 157., 155.],
       [202., 203., 204., ..., 127., 118., 107.]])

In [ ]:
from PIL import Image

image = apeend_train[0,:].reshape(28,28)
img_pil = Image.fromarray(np.uint8(image))

In [ ]:
image

array([[107., 118., 127., 134., 139., 143., 146., 150., 153., 156., 158.,
        160., 163., 165., 159., 166., 168., 170., 170., 171., 171., 171.,
        172., 171., 171., 170., 170., 169.],
       [111., 121., 129., 135., 141., 144., 148., 151., 154., 157., 160.,
        163., 164., 170., 119., 152., 171., 171., 170., 171., 172., 172.,
        172., 172., 172., 171., 171., 170.],
       [113., 123., 131., 137., 142., 145., 150., 152., 155., 158., 161.,
        163., 164., 172., 105., 142., 170., 171., 171., 171., 172., 172.,
        173., 173., 172., 171., 171., 171.],
       [116., 125., 133., 139., 143., 146., 151., 153., 156., 159., 162.,
        163., 167., 167.,  95., 144., 171., 172., 172., 172., 172., 172.,
        173., 173., 173., 172., 172., 171.],
       [117., 126., 134., 140., 145., 149., 153., 156., 158., 161., 163.,
        164., 175., 156.,  87., 154., 172., 173., 173., 173., 173., 173.,
        174., 174., 174., 173., 172., 172.],
       [119., 128., 136., 142., 146

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.RandomCrop(28)]
)

In [ ]:
img_pil

In [ ]:
transform(img_pil)

### 1. Data preparation

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ASL_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super(ASL_Dataset, self).__init__()

        train_df = pd.read_csv(path)
        y_train = train_df["label"]
        del train_df["label"]
        x_train = train_df.values

        self.x_train = x_train   # Nx784
        self.y_train = y_train
        self.transform = transform

    def __getitem__(self, index):
        x_train_sample = np.uint8( self.x_train[index].reshape(28,28) )

        if self.transform:
            x_train_pil = Image.fromarray(x_train_sample)
            x_train_np = np.array(self.transform(x_train_pil))
            x_train_tensor = torch.tensor( x_train_np[np.newaxis,:,:] ).float() # 1x28x28
        else:
            x_train_tensor = torch.tensor( x_train_sample[np.newaxis,:,:] ).float()

        y_train_tensor = torch.tensor( self.y_train[index] )

        return x_train_tensor, y_train_tensor

    def __len__(self):
        return self.x_train.shape[0]

In [ ]:
training_data = ASL_Dataset("sign_mnist_train.csv", transform=transform)
validation_data = ASL_Dataset("sign_mnist_valid.csv")

In [ ]:
train_loader = DataLoader(dataset=training_data, batch_size=1000, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=1000, shuffle=False)

### 2. Constructing a convolutional neural network

In [ ]:
import torch.nn as nn

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(32*7*7, 24)

    def forward(self, x):  ## 1x28x28
        x = self.conv1(x)   ## 16x28x28
        x = self.relu(x)
        x = self.pool(x)    ## 16x14x14

        x = self.conv2(x)   ## 32x14x14
        x = self.relu(x)
        x = self.pool(x)    ## 32x7x7

        x = self.conv3(x)   ## 32x7x7
        x = self.relu(x)

        x = x.view(-1, 32*7*7)
        x = self.fc(x)

        return x

In [ ]:
convnet = ConvNet()

### 3. Loss function and Optimization method

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. Training of a neural network 

In [ ]:
for epoch in range(20):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = convnet(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1   , Loss: 121.50997924804688
Epoch: 2   , Loss: 45.785685777664185
Epoch: 3   , Loss: 25.208059668540955
Epoch: 4   , Loss: 16.99350544810295
Epoch: 5   , Loss: 12.702322602272034
Epoch: 6   , Loss: 9.605162560939789
Epoch: 7   , Loss: 7.618293210864067
Epoch: 8   , Loss: 6.304530844092369
Epoch: 9   , Loss: 5.12280248105526
Epoch: 10   , Loss: 4.1271383464336395
Epoch: 11   , Loss: 3.4925658479332924
Epoch: 12   , Loss: 3.2638189494609833
Epoch: 13   , Loss: 3.013474978506565
Epoch: 14   , Loss: 2.4130427092313766
Epoch: 15   , Loss: 1.8893853425979614
Epoch: 16   , Loss: 1.5392857380211353
Epoch: 17   , Loss: 1.3828428611159325
Epoch: 18   , Loss: 1.3126205019652843
Epoch: 19   , Loss: 1.1317119896411896
Epoch: 20   , Loss: 1.1548319552093744


### 5. Prediction and evaludation for a test set

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred_test = convnet(inputs)
    pred_category = torch.argmax(pred_test, dim=1)

    pred_list = pred_list + list(pred_category)    
    label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Test accuracy:", accu)

Test accuracy: 0.7286670384829894
